In [1]:
import numpy as np
import pandas as pdcoding
import gzip

In [2]:
#SOMToolbox Parser
from SOMToolBox_Parse import SOMToolBox_Parse
idata = SOMToolBox_Parse("datasets/iris/iris.vec").read_weight_file()
weights = SOMToolBox_Parse("datasets/iris/iris.wgt.gz").read_weight_file()

In [3]:
#HitHistogram
def HitHist(_m, _n, _weights, _idata):
    hist = np.zeros(_m * _n)
    for vector in _idata: 
        position =np.argmin(np.sqrt(np.sum(np.power(_weights - vector, 2), axis=1)))
        hist[position] += 1

    return hist.reshape(_m, _n)

#U-Matrix - implementation
def UMatrix(_m, _n, _weights, _dim):
    U = _weights.reshape(_m, _n, _dim)
    U = np.insert(U, np.arange(1, _n), values=0, axis=1)
    U = np.insert(U, np.arange(1, _m), values=0, axis=0)
    #calculate interpolation
    for i in range(U.shape[0]): 
        if i%2==0:
            for j in range(1,U.shape[1],2):
                U[i,j][0] = np.linalg.norm(U[i,j-1] - U[i,j+1], axis=-1)
        else:
            for j in range(U.shape[1]):
                if j%2==0: 
                    U[i,j][0] = np.linalg.norm(U[i-1,j] - U[i+1,j], axis=-1)
                else:      
                    U[i,j][0] = (np.linalg.norm(U[i-1,j-1] - U[i+1,j+1], axis=-1) + np.linalg.norm(U[i+1,j-1] - U[i-1,j+1], axis=-1))/(2*np.sqrt(2))

    U = np.sum(U, axis=2) #move from Vector to Scalar

    for i in range(0, U.shape[0], 2): #count new values
        for j in range(0, U.shape[1], 2):
            region = []
            if j>0: region.append(U[i][j-1]) #check left border
            if i>0: region.append(U[i-1][j]) #check bottom
            if j<U.shape[1]-1: region.append(U[i][j+1]) #check right border
            if i<U.shape[0]-1: region.append(U[i+1][j]) #check upper border

            U[i,j] = np.median(region)

    return U

#SDH - implementation
def SDH(_m, _n, _weights, _idata, factor, approach):
    import heapq

    sdh_m = np.zeros( _m * _n)

    cs=0
    for i in range(factor): cs += factor-i

    for vector in _idata:
        dist = np.sqrt(np.sum(np.power(_weights - vector, 2), axis=1))
        c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
        if (approach==0): # normalized
            for j in range(factor):  sdh_m[c[j]] += (factor-j)/cs 
        if (approach==1):# based on distance
            for j in range(factor): sdh_m[c[j]] += 1.0/dist[c[j]] 
        if (approach==2): 
            dmin, dmax = min(dist[c]), max(dist[c])
            for j in range(factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(dmax-dmin)

    return sdh_m.reshape(_m, _n)


In [ ]:
import panel as pn
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

hithist = hv.Image(HitHist(weights['ydim'], weights['ydim'], weights['arr'], idata['arr'])).opts(xaxis=None, yaxis=None) 
um = hv.Image(UMatrix(weights['ydim'], weights['ydim'], weights['arr'], 4)).opts(xaxis=None, yaxis=None) 
sdh = hv.Image(SDH(weights['ydim'], weights['ydim'], weights['arr'], idata['arr'], 25, 0)).opts(xaxis=None, yaxis=None)   

hv.Layout([hithist.relabel('HitHist').opts(cmap='kr'), 
           um.relabel('U-Matrix').opts(cmap='jet'), sdh.relabel('SDH').opts(cmap='viridis')])

In [ ]:
import minisom

class AlignedSomLayer(minisom.MiniSom):
    """
    This class implements one layer of an aligned som.
    The key difference to the usual MiniSom is that it won't train on its own,
    but its training is directed by a parent (AlignedSom) using the update - scalded function.
    """
    def __init__(self, n, i, featuresA, x, y, input_len, sigma=1.0, learning_rate=0.5,
                 decay_function=minisom.asymptotic_decay,
                 neighborhood_function='gaussian', topology='rectangular',
                 activation_distance='euclidean', random_seed=42):
        """
        n: number of layers
        i: index of the layer
        featuresA: indices of the features that are in the A dataset
        all the other arguments get passed to the mini som init
        """
        super().__init__(x, y, input_len, sigma, learning_rate, decay_function, neighborhood_function, topology, activation_distance, random_seed)
        self.n = n
        self.i = i
        self.featuresA = featuresA
        
        self.underyling_activation_function = self._activation_distance
        self._activation_distance = self.activation_scale_wrapper
        
    def activation_scale_wrapper(self, x, w):
        """
        Wrapper around the activation function of the underlying som.
        Scales a vector x and the weights w by the factor given by i and w,
        then uses the same activation function as the underlying som would use.
        """
        n = self.n
        i = self.i
        scaleA = (n-i-1)/(n-1)
        scaleB = i/(n-1)

        factor = []
        for j in range(self._input_len):
            if j in self.featuresA:
                factor.append(scaleA)
            else:
                factor.append(scaleB)
        
        xcopy = x.copy()
        wcopy = w.copy()

        xcopy *= np.array(factor)
        wcopy *= np.array(factor)

        return self.underyling_activation_function(xcopy, wcopy)


    def update_scaled(self, x, win, t, max_iteration, scaling_factor):
        """
        Updates the weights of the neurons. Scaled by the scaling Factor.
        In practice, this factor can be something like 1 / how far the layer is from the "pivot" layer
        x : np.array
            Current pattern to learn.
        win : tuple
            Position of the winning neuron for x (array or tuple).
        t : int
            rate of decay for sigma and learning rate
        max_iteration : int
            If use_epochs is True:
                Number of epochs the SOM will be trained for
            If use_epochs is False:
                Maximum number of iterations (one iteration per sample).
        scaling_factor: float
            factor the update is scaled by
        """
        eta = self._decay_function(self._learning_rate, t, max_iteration)
        # sigma and learning rate decrease with the same rule
        sig = self._decay_function(self._sigma, t, max_iteration)
        # improves the performances
        g = self.neighborhood(win, sig)*eta*scaling_factor
        # w_new = eta * neighborhood_function * (x-w)
        self._weights += np.einsum('ij, ijk->ijk', g, x-self._weights)

    # def update_3d(self, x, win, t, max_iteration, chosen_layer_index):
    #     """Updates the weights of the neurons.

    #     Parameters
    #     ----------
    #     x : np.array
    #         Current pattern to learn.
    #     win : tuple
    #         Position of the winning neuron for x (array or tuple).
    #     t : int
    #         rate of decay for sigma and learning rate
    #     max_iteration : int
    #         If use_epochs is True:
    #             Number of epochs the SOM will be trained for
    #         If use_epochs is False:
    #             Maximum number of iterations (one iteration per sample).
    #     """
    #     win_3d = win


    #     eta = self._decay_function(self._learning_rate, t, max_iteration)
    #     # sigma and learning rate decrease with the same rule
    #     sig = self._decay_function(self._sigma, t, max_iteration)
    #     # improves the performances
    #     g = self.neighborhood(win, sig)*eta
    #     # w_new = eta * neighborhood_function * (x-w)
    #     self._weights += np.einsum('ij, ijk->ijk', g, x-self._weights)

    # def _cosine_distance(self, x, w):
    #     num = (w * x).sum(axis=2)
    #     denum = multiply(linalg.norm(w, axis=2), linalg.norm(x))
    #     return 1 - num / (denum+1e-8)

    # def _euclidean_distance(self, x, w):
    #     return linalg.norm(subtract(x, w), axis=-1)

    # def _manhattan_distance(self, x, w):
    #     return linalg.norm(subtract(x, w), ord=1, axis=-1)

    # def _chebyshev_distance(self, x, w):
    #     return max(subtract(x, w), axis=-1)


# import minisom
class AlignedSom:
    def __init__(self, x, y, input_len, n, featuresA, base_scaling_factor=1.0, sigma=1.0, learning_rate=0.5,
                 decay_function=minisom.asymptotic_decay,
                 neighborhood_function='gaussian', topology='rectangular',
                 activation_distance='euclidean', random_seed=42):
        """
        initializes the Aligned Som class.
        n guides how many maps are generated
        featuresA sets which features are in the A set (The rest is in B). This should be a list of indices < input_len

        All the other params get handed over to the "children soms"
        """
        self.n = n
        self.featuresA = featuresA
        self.x = x
        self.y = y
        self._random_seed = random_seed

        self._learning_rate = learning_rate
        self._sigma = sigma
        self._input_len = input_len

        self.topology = topology

        self._decay_function = decay_function
        
        self._random_generator = np.random.RandomState(random_seed)

        self._layers = [AlignedSomLayer(n, i, featuresA, x, y, input_len, sigma=sigma, learning_rate=learning_rate, 
                                        decay_function=decay_function, 
                                        neighborhood_function=neighborhood_function, topology=topology, 
                                        activation_distance=activation_distance, random_seed=random_seed
                                        ) for i in range(n)]
        
        self.base_scaling_factor = base_scaling_factor
        self._weights = [self._layers[i]._weights for i in range(n)]


    def train(self, data, num_iteration,
              random_order=False, verbose=False, use_epochs=False):
        """
        Trains the SOM.

        Parameters
        ----------
        data : np.array or list
            Data matrix.

        num_iteration : int
            If use_epochs is False, the weights will be
            updated num_iteration times. Otherwise they will be updated
            len(data)*num_iteration times.

        random_order : bool (default=False)
            If True, samples are picked in random order.
            Otherwise the samples are picked sequentially.

        verbose : bool (default=False)
            If True the status of the training will be
            printed each time the weights are updated.

        use_epochs : bool (default=False)
            If True the SOM will be trained for num_iteration epochs.
            In one epoch the weights are updated len(data) times and
            the learning rate is constat throughout a single epoch.
        """
        
    
        # self._check_iteration_number(num_iteration)
        # self._check_input_len(data)

        random_generator = None
        if random_order:
            random_generator = self._random_generator
        iterations = minisom._build_iteration_indexes(len(data), num_iteration,
                                              verbose, random_generator,
                                              use_epochs)
        if use_epochs:
            def get_decay_rate(iteration_index, data_len):
                return int(iteration_index / data_len)
        else:
            def get_decay_rate(iteration_index, data_len):
                return int(iteration_index)

        for t, iteration in enumerate(iterations):
            #picks a random layer
            chosen_layer_index = self._random_generator.randint(low=0, high=self.n)
            decay_rate = get_decay_rate(t, len(data))
            
            cur_training_sample = data[iteration]
            #calculates winner (a pair of indices)
            winner = self._layers[chosen_layer_index].winner(cur_training_sample)


            for update_layer_index in range(self.n):
                #calculates the distance to the current layer
                diff = chosen_layer_index - update_layer_index
                if diff < 0:
                    diff = -diff

                scaling_factor = self.base_scaling_factor/(1+diff)
                self._layers[update_layer_index].update_scaled(cur_training_sample, winner,
                        decay_rate, num_iteration, scaling_factor)
                # self._layers[update_layer_index].update_3d(cur_training_sample, winner,
                #         decay_rate, num_iteration, chosen_layer_index)
                
        
        # for t, iteration in enumerate(iterations):
        #     decay_rate = get_decay_rate(t, len(data))
        #     self.update(data[iteration], self.winner(data[iteration]),
        #                 decay_rate, num_iteration)
        if verbose:
            print('\n quantization error:', self.quantization_error(data))


In [6]:
alignedSom = AlignedSom(2, 2, 10, 11, [0, 1, 2])

data = [[1]*10,[2]*10]
alignedSom.train(data, 10)

In [7]:

class AlignedSomVis():
    def __init__(self, weights, input_data, chosen_visulization_index=0):
        """Initializes an Aligned SOM Visulization object.

        Parameters
        ----------
        weights : list
            a list of arrays containing the weights of the SOM layers.

        input_data: np.array
            the array containing the data vectors

        chosen_visulization_index : int
            the index for the default visulization shown by _mainview.
            Possible values: 0 for Hit Histogram, 1 for U-matrix or 2 for SDH.
            Default value is 0.
        """
        self._idata = input_data
        self._num_layers = len(weights)
        
        xdim = weights[0].shape[0]
        ydim = weights[0].shape[1]
        self._weights = [np.reshape(weights[i], (xdim*ydim, -1)) for i in range(self._num_layers)]
        
        vector_dim = len(self._idata[0])
        
        self._images_HitHist = [hv.Image(HitHist(xdim, ydim, self._weights[i], self._idata)).opts(xaxis=None, yaxis=None) for i in range(self._num_layers)]
        self._images_UMatrix = [hv.Image(UMatrix(xdim, ydim, self._weights[i], vector_dim)).opts(xaxis=None, yaxis=None) for i in range(self._num_layers)]
        self._images_SDH = [hv.Image(SDH(xdim, ydim, self._weights[i], self._idata, 25, 0)).opts(xaxis=None, yaxis=None) for i in range(self._num_layers)]
        
        self._maps_SDH = [i for i in range(self._num_layers)]
        sdh_paramd = [lambda factor, approach: hv.Image(SDH(xdim, ydim, self._weights[i], self._idata, factor, approach)).opts(xaxis=None, yaxis=None) for i in range(self._num_layers)]

        self._maps_SDH[0] =  hv.HoloMap({(factor, approach): sdh_paramd[0](factor, approach) for factor in [i+1 for i in range(100)] for approach in [0, 1, 2]},  kdims=['factor', 'approach'])
        for i in range(1, self._num_layers):
            self._maps_SDH[i] = hv.DynamicMap(sdh_paramd[i], kdims=['factor', 'approach'])
        ## NOTE: could not get the code below to run, so used the holomap above as a base
        # for dmap in self._maps_SDH:                                                   
        #     dmap.redim.values(factor=[i+1 for i in range(100)], approach=[0, 1, 2])

        self._visualizations = [self._images_HitHist, self._images_UMatrix, self._maps_SDH]
        self._default_color_map = ['kr', 'jet', 'viridis']
        self._titles = ['Hit Histogram', 'U-matrix', 'Smoothed Data Histograms']

        implemented_vis_number = len(self._visualizations)

        self._available_visulizations = [hv.Layout([self._visualizations[j][i].relabel(f'Layer {i}').opts(cmap=self._default_color_map[j]) for i in range(self._num_layers)]).opts(title=self._titles[j]) for j in range(implemented_vis_number)]
        self._mainview = self._available_visulizations[chosen_visulization_index]

C) Evaluation Report
1) Perform and document (!) the testing of the components you coded by defining and
evaluating suitable tests to evaluate the correctness and robustness of the coded modules.

2) For systematic evaluation of tasks a-h, pick the Chainlink Data Set and the 10-Clusters
dataset from
http://www.ifs.tuwien.ac.at/dm/somtoolbox/datasets.html 						--> already available in pysomvis datasets

3) Train a 10x10 (small) and a 100x60 (large) SOM. Make sure that the SOMs are properly
trained, i.e. that the structures to be expected in the SOM become clearly visible by identifying
suitable parameters for the initial neighborhood radius and initial learning rate.

In [8]:
# code goes here
from minisom import MiniSom
# from pysomvis import PySOMVis #

idata_chainlink   = SOMToolBox_Parse("datasets/chainlink/chainlink.vec").read_weight_file()
idata_10clusters   = SOMToolBox_Parse("datasets/10clusters/10clusters.vec").read_weight_file()

A_set_chainlink = [0, 1]
A_set_10clusters = [0, 2, 4, 6, 8]

num_layers_chainlink = 10
num_layers_10clusters = 10

small_som_chainlink = AlignedSom(10, 10, 3, num_layers_chainlink, A_set_chainlink, sigma=7, learning_rate=0.7)
# small_som_chainlink = MiniSom(10, 10, 3, sigma=7, learning_rate=0.7)
small_som_chainlink.train(idata_chainlink['arr'], 10000)
# large_som_chainlink = MiniSom(100, 60, 3, sigma=7, learning_rate=0.7)
# large_som_chainlink.train(idata_chainlink['arr'], 10000)



In [ ]:
vis = AlignedSomVis(weights=small_som_chainlink._weights, input_data=idata_chainlink['arr'])
vis._mainview
vis._available_visulizations[2]
# vis = AlignedSomVisFirstAttempt(weights=small_som_chainlink._weights, input_data=idata_chainlink['arr'])
# vis._mainview
# vis = PySOMVis(weights=small_som_chainlink._weights, input_data=idata_chainlink['arr'])
# vis._mainview
# vis = PySOMVis(weights=large_som_chainlink._weights, input_data=idata_chainlink['arr'])
# vis._mainview

# small_som_10clusters = MiniSom(10, 10, 10, learning_rate=0.7)
# small_som_10clusters.train(idata_10clusters['arr'], 1000)
# large_som_10clusters = MiniSom(100, 60, 10, learning_rate=0.7)
# large_som_10clusters.train(idata_10clusters['arr'], 1000)



4) Show the visualizations, providing examples with different parameter settings and
comparisons that allow a validation of the correctness of the implementation. Specifically,
test a few extreme values for the parameter settings.

5) Where an identical visualization exists in the JÁVA SOM toolbox, read a SOM pre-trained
with the JAVA SOM Toolbox (import functions are provided in the notebook) and compare
your visualization with the one produced by the Java SOMToolbox (using either the pre
trained SOMs provided with the toolbox, or any that your colleagues who do the analytics
option of the exercise share with you). --> aligned SOM not part of JAVA SOM Toolbox

6) Provide (export/print) the notebook as separate PDF report that comprises all information.
Hence, the PDF export of the report needs to contain the fully-computed notebook with the
according visualizations shown as results and the information that can be derived from the
visualizations clearly described and semantically analyzed. Make sure that each visualization
includes the parameter setting applied. Specifically, the PDF export needs to contain:
- the implementation developed, explaining key parts of the code of each cell.
- the way the code was systematically tested for correctness, including the test cases
as part of the notebook.
- the evaluations performed under item 3) above, demonstrating the correctness of the
implementation, and the information gained.
- Where applicable: Comparison of the visualization with the identical visualizations
(reading the same trained SOM files) using the SOM Java Toolbox


In [ ]:
#probably some more code
!which xelatex